In [1]:
import nltk
import re
from pymystem3 import Mystem

In [2]:
sentences = ['Вася читает мою книгу', 'Напиши какое-нибудь письмо', \
             'Этот весёлый мальчик идёт', 'Он любит читать всякие книги']

In [3]:
rules = """
    S -> NP VP
    S -> VP
    NP -> ADJ NP
    NP -> ADJ N
    VP -> V NP
    VP -> V VP
    NP -> NN
    NP -> N
    VP -> V
    N -> 'книга' | 'письмо' | 'мальчик' | 'он'
    NN -> 'вася'
    V -> 'читать' | 'написать' | 'идти' | 'любить'
    ADJ -> 'мой' | 'какой-нибудь' | 'этот' | 'весёлый' | 'всякий'
""".split('\n')

In [4]:
grammar = nltk.CFG.fromstring('\n'.join(rules))

# Анализ предложения с применением алгоритма CYK.

|S|||||
|------|------|------|------|------|
||VP||||
|------|------|------|------|------|
|||VP|||
|------|------|------|------|------|
|NP|||NP||
|------|------|------|------|------|
|N|V|V|ADJ|N|
|------|------|------|------|------|
|Он|любит|читать|всякие|книги|

0. N -> 'он', V -> 'любит', V -> 'читать', ADJ -> 'всякие', 'N' -> 'книги'
1. NP -> N (он),  NP -> ADJ N (всякие книги)
2. VP -> V NP (читать всякие книги)
3. VP -> V VP (любит читать всякие книги)
4. S -> NP VP (он любит читать всякие книги)

# Анализ предложения с применением алгоритма Эрли

"Он любит читать всякие книги" 

1. [0:0] S -> * NP VP
2. [0:0] NP -> * N
3. [0:0] N -> * 'он'
4. [0:1] N -> 'он' * 
5. [0:1] NP -> 'он' * 
6. [0:1] S -> NP * VP
7. [1:1] VP -> * V VP
8. [1:1] V -> * 'любит'
9. [1:2] V -> 'любит' * 
10. [1:2] VP -> V * VP
11. [2:2] VP -> * V NP
12. [2:2] V -> * 'читать'
13. [2:3] V -> 'читать' * 
14. [2:3] VP -> V * NP
15. [3:3] NP -> * ADJ N
16. [3:3] ADJ -> * 'всякие'
17. [3:4] ADJ -> 'всякие' * 
18. [3:4] NP -> ADJ * N
19. [4:4] N -> * 'книги'
20. [4:5] N -> 'книги' * 
21. [3:5] NP -> ADJ N * 
22. [2:5] VP -> V NP * 
23. [1:5] VP -> V VP * 
24. [0:5] S -> NP VP * 

In [5]:
cp = nltk.EarleyChartParser(grammar, trace=1)
m = Mystem()

In [6]:
grammar.productions()

[S -> NP VP,
 S -> VP,
 NP -> ADJ NP,
 NP -> ADJ N,
 VP -> V NP,
 VP -> V VP,
 NP -> NN,
 NP -> N,
 VP -> V,
 N -> 'книга',
 N -> 'письмо',
 N -> 'мальчик',
 N -> 'он',
 NN -> 'вася',
 V -> 'читать',
 V -> 'написать',
 V -> 'идти',
 V -> 'любить',
 ADJ -> 'мой',
 ADJ -> 'какой-нибудь',
 ADJ -> 'этот',
 ADJ -> 'весёлый',
 ADJ -> 'всякий']

In [7]:
def print_parses(parser, sentence):
    sentence = m.lemmatize(sentence)[::2]
    for tree in parser.parse(sentence):
        print(tree)

In [8]:
print_parses(cp, sentences[3]) # всё работает!

|.   он  . любить. читать. всякий. книга .|
|[-------]       .       .       .       .| [0:1] 'он'
|.       [-------]       .       .       .| [1:2] 'любить'
|.       .       [-------]       .       .| [2:3] 'читать'
|.       .       .       [-------]       .| [3:4] 'всякий'
|.       .       .       .       [-------]| [4:5] 'книга'
|>       .       .       .       .       .| [0:0] S  -> * NP VP
|>       .       .       .       .       .| [0:0] S  -> * VP
|>       .       .       .       .       .| [0:0] VP -> * V NP
|>       .       .       .       .       .| [0:0] VP -> * V VP
|>       .       .       .       .       .| [0:0] VP -> * V
|>       .       .       .       .       .| [0:0] NP -> * ADJ NP
|>       .       .       .       .       .| [0:0] NP -> * ADJ N
|>       .       .       .       .       .| [0:0] NP -> * NN
|>       .       .       .       .       .| [0:0] NP -> * N
|>       .       .       .       .       .| [0:0] N  -> * 'он'
|[-------]       .       .       .       .

In [9]:
def get_vocab(rules): #функция, вытаскивающая из грамматики список слов (я не нашла метода для этого в NLTK :( ))
    ans = []
    for rule in rules:
        if re.search("'(.*?)'", rule):
            for i in re.findall("'(.*?)'", rule):
                ans.append(i)
    return ans

In [10]:
get_vocab(rules)

['книга',
 'письмо',
 'мальчик',
 'он',
 'вася',
 'читать',
 'написать',
 'идти',
 'любить',
 'мой',
 'какой-нибудь',
 'этот',
 'весёлый',
 'всякий']

In [11]:
def parse_with_unknown_words(rules, sentence): #разбираемся с неизвестными словами
    sentence = m.lemmatize(sentence)[::2]
    vocab = get_vocab(rules)
    pos_dict = {'S': 'N', 'V': 'V', 'A': 'ADJ', 'APRO': 'ADJ'}
    for word in sentence:
        if word.lower() not in vocab:
            #если встретилось неизвестное слово, находим его часть речи и создаём новое правило
            pos = m.analyze(word)[0]['analysis'][0]['gr'].split('=')[0].split(',')[0]
            if pos in pos_dict:
                rules.append(f"{pos_dict[pos]} -> '{word}'")
            else:
                raise ValueError('A word with unknown POS has appeared!')
    grammar = nltk.CFG.fromstring('\n'.join(rules))
    parser = nltk.EarleyChartParser(grammar, trace=1)
    for tree in parser.parse(sentence):
        print(tree)

In [12]:
parse_with_unknown_words(rules, 'Вася читает мой журнал')

|.   вася  .  читать .   мой   .  журнал .|
|[---------]         .         .         .| [0:1] 'вася'
|.         [---------]         .         .| [1:2] 'читать'
|.         .         [---------]         .| [2:3] 'мой'
|.         .         .         [---------]| [3:4] 'журнал'
|>         .         .         .         .| [0:0] S  -> * NP VP
|>         .         .         .         .| [0:0] S  -> * VP
|>         .         .         .         .| [0:0] VP -> * V NP
|>         .         .         .         .| [0:0] VP -> * V VP
|>         .         .         .         .| [0:0] VP -> * V
|>         .         .         .         .| [0:0] NP -> * ADJ NP
|>         .         .         .         .| [0:0] NP -> * ADJ N
|>         .         .         .         .| [0:0] NP -> * NN
|>         .         .         .         .| [0:0] NP -> * N
|>         .         .         .         .| [0:0] NN -> * 'вася'
|[---------]         .         .         .| [0:1] NN -> 'вася' *
|[---------]         .         .  